need to use colab for gpus

1. Setup and Load Data

1.1 Install Dependencies and Setup

In [12]:
import os
import cv2
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
import torchvision.models.segmentation
import torch
import torchvision.transforms as T
from torch.utils.data import TensorDataset, DataLoader
from PIL import Image
import scipy.io
from sklearn.model_selection import train_test_split
import random
from torchviz import make_dot

# Set the device to use
# CUDA refers to the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Hyperparameters
num_epochs = 10
num_classes = 10  # there are 10 digits: 0 to 9
batch_size = 256

## Fixing Random Seed for Reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)


# If you are on CoLab and successfully using the GPU, this print should
#   contain "cuda" in it
print(str(device))
assert('cuda' in str(device))  # comment out this assert if you are not using a GPU

cuda:0


In [13]:
x = cv2.cvtColor(cv2.imread('trainingdataimages/gentrain_001.png'), cv2.COLOR_BGR2RGB)
x

array([[[ 99, 204, 111],
        [ 99, 204, 111],
        [ 99, 204, 111],
        ...,
        [ 43, 195, 167],
        [ 43, 195, 167],
        [ 43, 195, 167]],

       [[ 99, 204, 111],
        [ 99, 204, 111],
        [ 99, 204, 111],
        ...,
        [ 43, 195, 167],
        [ 43, 195, 167],
        [ 43, 195, 167]],

       [[ 99, 204, 111],
        [ 99, 204, 111],
        [ 99, 204, 111],
        ...,
        [ 43, 195, 167],
        [ 43, 195, 167],
        [ 43, 195, 167]],

       ...,

       [[ 39, 150, 235],
        [ 39, 150, 235],
        [ 39, 150, 235],
        ...,
        [ 45, 134, 246],
        [ 45, 134, 246],
        [ 45, 134, 246]],

       [[ 39, 150, 235],
        [ 39, 150, 235],
        [ 39, 150, 235],
        ...,
        [ 45, 134, 246],
        [ 45, 134, 246],
        [ 45, 134, 246]],

       [[ 39, 150, 235],
        [ 39, 150, 235],
        [ 39, 150, 235],
        ...,
        [ 45, 134, 246],
        [ 45, 134, 246],
        [ 45, 134, 246]]

In [14]:
image = Image.open('trainingdataimages/gentrain_001.png')
data = np.asarray(image)
data

array([[[ 99, 204, 111],
        [ 99, 204, 111],
        [ 99, 204, 111],
        ...,
        [ 43, 195, 167],
        [ 43, 195, 167],
        [ 43, 195, 167]],

       [[ 99, 204, 111],
        [ 99, 204, 111],
        [ 99, 204, 111],
        ...,
        [ 43, 195, 167],
        [ 43, 195, 167],
        [ 43, 195, 167]],

       [[ 99, 204, 111],
        [ 99, 204, 111],
        [ 99, 204, 111],
        ...,
        [ 43, 195, 167],
        [ 43, 195, 167],
        [ 43, 195, 167]],

       ...,

       [[ 39, 150, 235],
        [ 39, 150, 235],
        [ 39, 150, 235],
        ...,
        [ 45, 134, 246],
        [ 45, 134, 246],
        [ 45, 134, 246]],

       [[ 39, 150, 235],
        [ 39, 150, 235],
        [ 39, 150, 235],
        ...,
        [ 45, 134, 246],
        [ 45, 134, 246],
        [ 45, 134, 246]],

       [[ 39, 150, 235],
        [ 39, 150, 235],
        [ 39, 150, 235],
        ...,
        [ 45, 134, 246],
        [ 45, 134, 246],
        [ 45, 134, 246]]

theta: ([,], [0,3], [0, 1])

In [15]:
import tqdm

image_list = []
theta_list = []

directory = 'trainingdataimages'
 
# iterate over files in
# that directory
for filename in tqdm.tqdm(os.listdir(directory)):
    # print(filename)
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        # Opens a image in RGB mode
        image = Image.open(f)
        data = np.asarray(image)
        data = cv2.resize(data, (500,500), interpolation=cv2.INTER_LINEAR)
        matfile = 'trainingdatavalues/' + filename[:len(filename) - 4] + '.mat'
        mat = scipy.io.loadmat(matfile)
        image_list.append(data)
        theta_list.append(mat['th0'][0])

print(image_list[0].shape)
print(theta_list[0])

theta_list = np.asarray(theta_list)
image_list = np.asarray(image_list)

# print(theta_list)

# print(image_list.shape)

image_train, image_test, theta_train, theta_test = train_test_split(image_list, theta_list)

print(image_train.shape)

image_list = torch.tensor(image_list).float()
# print(image_list)
theta_list = torch.tensor(theta_list)
# print(theta_list)

# image_list.shape

100%|██████████| 90/90 [00:00<00:00, 196.45it/s]


(500, 500, 3)
[1.6e+06 2.5e+00 2.0e+04]
(67, 500, 500, 3)


In [16]:
# normalize data:

# images
imean = torch.std(image_list, dim=0)
istd = torch.mean(image_list, dim=0)
image_tensor = torch.div(torch.sub(image_list, imean), istd)
print(image_tensor)

# theta
tmean = torch.std(theta_list, dim=0)
tstd = torch.mean(theta_list, dim=0)
theta_tensor = torch.div(torch.sub(theta_list, tmean), tstd)
print(theta_tensor)

tensor([[[[ 0.5668,  0.9585,  0.2730],
          [ 0.5927,  0.9632,  0.2735],
          [ 0.5927,  0.9632,  0.2735],
          ...,
          [-0.2026,  0.9445,  0.5992],
          [-0.2053,  0.9431,  0.6005],
          [-0.2054,  0.9429,  0.6005]],

         [[ 0.5927,  0.9633,  0.2735],
          [ 0.6038,  0.9665,  0.2733],
          [ 0.6038,  0.9665,  0.2733],
          ...,
          [-0.2103,  0.9450,  0.5980],
          [-0.2132,  0.9438,  0.5995],
          [-0.2133,  0.9434,  0.5995]],

         [[ 0.5927,  0.9632,  0.2735],
          [ 0.6038,  0.9665,  0.2733],
          [ 0.6038,  0.9665,  0.2733],
          ...,
          [-0.2103,  0.9450,  0.5980],
          [-0.2134,  0.9436,  0.5993],
          [-0.2133,  0.9434,  0.5995]],

         ...,

         [[-0.3078,  0.6702,  1.0589],
          [-0.3202,  0.6733,  1.0613],
          [-0.3202,  0.6733,  1.0613],
          ...,
          [-0.3211,  0.5814,  1.0410],
          [-0.3313,  0.5828,  1.0431],
          [-0.3322,  0

In [17]:
dataset = TensorDataset(image_list, theta_list)
print(dataset)
train_loader = DataLoader(dataset, batch_size=2, shuffle=False)
print(train_loader)

In [18]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2) 
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2) 
        self.pool2 = nn.MaxPool2d(2, 2) 

        self.size_linear = 64*125*125
        self.fc1 = nn.Linear(self.size_linear, 512)
        self.fc2 = nn.Linear(512, 10) # ACT6

    def forward(self, x):
            x = self.pool1(F.relu(self.conv1(x))) # we provide this line for you
            x = self.pool2(F.relu(self.conv2(x))) # ACT7: apply the second convolution and max pooling
            x = x.view(-1, self.size_linear) # this flattens x into a 1D vector
            x = F.relu(self.fc1(x)) # ACT8: Apply the first fully connected layer and ReLU
            x = self.fc2(x) # ACT9: Apply the second fully connected layer (no ReLU)
            return x
    
##ASSERT: checks if your CNN has the correct output shape
with torch.no_grad():  # tells PyTorch not to track gradients here
    # test_data is 100 random images, 1 channel, 28-by-28
    test_data = torch.rand(100,3,500,500)
    test_net = ConvNet()
    out = test_net.forward(test_data)
    # the output should have size (100,10)
    assert(out.size()==(100,10))

In [19]:
# test_data = torch.rand(100,3,500,500)
# test_net = ConvNet()
# out = test_net.forward(test_data)
# make_dot(out)  # ACT10

In [20]:
### ACT11-ACT15 Please fill the code below
class Trainer():
    def __init__(self,net=None,optim=None,loss_function=None, train_loader=None):
        self.net = net
        self.optim = optim
        self.loss_function = loss_function
        self.train_loader = train_loader

    def train(self,epochs):
        losses = []
        for epoch in range(epochs):
            epoch_loss = 0.0
            epoch_steps = 0
            for data in self.train_loader:
                
                # Moving this batch to GPU
                # Note that X has shape (batch_size, number of channels, height, width)
                # which is equal to (256,1,28,28) since our default batch_size = 256 and 
                # the image has only 1 channel
                X = data[0].to(device)
                y = data[1].to(device)
                
                # ACT11-Zero the gradient in the optimizer i.e. self.optim
                ################
                # Your code here
                self.optim.zero_grad()
                ################

                # ACT12-Getting the output of the Network
                ################
                # Your code here
                out = self.net(X)
                ################

                # ACT13-Computing loss using loss function i.e. self.loss_function
                ################
                # Your code here
                loss = self.loss_function(out, y)
                ################

                # ACT14-Backpropagate to compute gradients of parameteres
                ################
                # Your code here
                loss.backward()
                ################

                # ACT15-Call the optimizer i.e. self.optim
                ################
                # Your code here
                self.optim.step()
                ################

                epoch_loss += loss.item()
                epoch_steps += 1
            # average loss of epoch
            losses.append(epoch_loss / epoch_steps)
            print("epoch [%d]: loss %.3f" % (epoch+1, losses[-1]))
        return losses